### MobileNet_v1

**创新点**

1. 深度可分离卷积

2. 提供两个超参数控制特征图通道以及特征图尺寸

In [16]:
import torch
from torch import nn
import torch.nn.functional as F

In [17]:
'''
标准卷积类
'''

class Conv_bn(nn.Module):
    def __init__(self,in_dim,out_dim,stride):
        super(Conv_bn,self).__init__()
        self.conv_bn=nn.Sequential(nn.Conv2d(in_dim,out_dim,kernel_size=3,stride=stride,padding=1,bias=False),
                                  nn.BatchNorm2d(out_dim),
                                  nn.ReLU(True))
    def forward(self,x):
        return self.conv_bn(x)

In [18]:
'''
深度可分离卷积类
'''

class Conv_dw(nn.Module):
    def __init__(self,in_dim,out_dim,stride):
        super(Conv_dw,self).__init__()
        self.conv_dw=nn.Sequential(nn.Conv2d(in_dim,in_dim,kernel_size=3,stride=stride,padding=1,groups=in_dim,bias=False),
                                  nn.BatchNorm2d(in_dim),
                                  nn.ReLU6(True),
                                  nn.Conv2d(in_dim,out_dim,kernel_size=1,stride=1,padding=0,bias=False),
                                  nn.BatchNorm2d(out_dim),
                                  nn.ReLU6(True))
    def forward(self,x):
        return self.conv_dw(x)

In [19]:
class MobileNet_v1(nn.Module):
    def __init__(self):
        super(MobileNet_v1, self).__init__()

        layers = []
        layers += [Conv_bn(3, 32, 2),Conv_dw(32,64,1)]
        in_planes = 64
        for i in range(3):
            layers += [Conv_dw(in_planes, 2 * in_planes,2),Conv_dw(2*in_planes,2*in_planes,1)]
            in_planes *= 2
        for i in range(4):
            layers += [Conv_dw(512, 512, 1)]

        layers += [
            Conv_dw(512, 1024, 2),
            Conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7)
        ]

        self.feature = nn.Sequential(*layers)
        self.classifier = nn.Linear(1024, 1000)

    def forward(self, x):
        out = self.feature(x)
        out = out.reshape(-1, 1024)
        out = self.classifier(out)
        y = F.softmax(out, dim=1)
        return y

In [20]:
model=MobileNet_v1()

In [21]:
model

MobileNet_v1(
  (feature): Sequential(
    (0): Conv_bn(
      (conv_bn): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
      )
    )
    (1): Conv_dw(
      (conv_dw): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
        (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU6(inplace)
      )
    )
    (2): Conv_dw(
      (conv_dw): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [28]:
x=torch.randn(1,3,224,224)
y=model(x)
y.shape

torch.Size([1, 1000])

### MobileNet_v2

**创新点**

    1. 采用残差网络模型
    
    2. 去掉每个block最后一层的ReLU6，保持特征

In [27]:
class MobileNet_v2(nn.Module):
    def __init__(self, in_dim, out_dim, stride,
                 expand_ratio):  # expand_ratio 中间层扩展数量
        super(MobileNet_v2, self).__init__()
        self.stride = stride
        # 中间通道计算
        hidden_dim = round(expand_ratio * in_dim)

        # 1*1卷积模块
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim,
                      hidden_dim,
                      kernel_size=1,
                      stride=1,
                      padding=0,
                      bias=False), nn.BatchNorm2d(hidden_dim), nn.ReLU6(True))

        # 3*3 深度卷积模块
        self.conv_dw = nn.Sequential(
            nn.Conv2d(hidden_dim,
                      hidden_dim,
                      kernel_size=3,
                      stride=stride,
                      padding=1,
                      groups=hidden_dim,
                      bias=False), nn.BatchNorm2d(hidden_dim), nn.ReLU6(True))

        # 1*1 线性层
        self.linear = nn.Sequential(
            nn.Conv2d(hidden_dim, out_dim, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(out_dim), nn.ReLU6(True))

    def forward(self, x):
        out = self.conv(x)
        out = self.conv_dw(out)
        out = self.linear(out)
        if self.stride == 2:
            return out
        else:
            return out + x

In [28]:
model_=MobileNet_v2(24,24,1,6)

In [29]:
model_

MobileNet_v2(
  (conv): Sequential(
    (0): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace)
  )
  (conv_dw): Sequential(
    (0): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=144, bias=False)
    (1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace)
  )
  (linear): Sequential(
    (0): Conv2d(144, 24, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace)
  )
)

In [30]:
x=torch.randn(1,24,56,56)
y=model_(x)
y.shape

torch.Size([1, 24, 56, 56])